# PROJETO INTEGRADOR III - Modelo de aprendizagem de máquina

### Parâmetros que podem ser usados para definir a aprovação:

- nota (de cada área de conhecimento e com peso, mesmo que todos sejam iguais a 1);
- cotas (vou precisar tratar isso aqui);
- notas de corte;
- curso (vou precisar tratar isso aqui);
- universidade (vou precisar tratar isso aqui: campus, IES ou o que?);
- Local da IES;
- local do candidato;


### feature engineering

pontos importantes para serem destacados:

- *notas*: tratar os dados que estão fora da distribuição normal;
- *cotas, curso e universidade*: considerar usar um target encoding ou count encoding;
- *notas de corte*: talvez não será necessário passar por um tratamento;
- *locais*: pode ser usado um label encoding, mas é necessário avaliar o contexto. oneHotEncoding seria muito custoso ( 27 * 2 + 54 = 108 colunas).


### Avaliação de modelo de linguagem
precisão, recall, acurácia e área sob a curva ROC (Receiver Operating Characteristic).


In [65]:
import pandas as pd
import psycopg2 as pspg
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter 
from scipy import stats
import dask.dataframe as dd
import sqlalchemy as sql
from category_encoders.target_encoder import TargetEncoder
from category_encoders.count import CountEncoder
from sklearn.preprocessing import (OrdinalEncoder, OneHotEncoder, Normalizer, StandardScaler )
CONN = pspg.connect(database='notas', user='dev', password='dev', host='localhost', port='5030')

In [66]:
df = pd.read_sql('''
  select 
    * from sisu_data
  tablesample bernoulli(2)
  ''', con=CONN)

/tmp/ipykernel_7699/3420351349.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''


In [67]:
df.columns

Index(['ano', 'edicao', 'codigo_etapa', 'etapa', 'codigo_ies', 'nome_ies',
       'sigla_ies', 'uf_ies', 'codigo_campus', 'nome_campus', 'uf_campus',
       'municipio_campus', 'codigo_curso', 'nome_curso', 'grau', 'turno',
       'ds_periodicidade', 'tp_cota', 'tipo_mod_concorrencia',
       'mod_concorrencia', 'qt_vagas_concorrencia', 'percentual_bonus',
       'peso_l', 'peso_ch', 'peso_cn', 'peso_m', 'peso_r', 'nota_minima_l',
       'nota_minima_ch', 'nota_minima_cn', 'nota_minima_m', 'nota_minima_r',
       'media_minima', 'cpf', 'inscricao_enem', 'inscrito', 'sexo',
       'data_nascimento', 'uf_candidato', 'municipio_candidato', 'opcao',
       'nota_l', 'nota_ch', 'nota_cn', 'nota_m', 'nota_r', 'nota_l_com_peso',
       'nota_ch_com_peso', 'nota_cn_com_peso', 'nota_m_com_peso',
       'nota_r_com_peso', 'nota_candidato', 'nota_corte', 'classificacao',
       'aprovado', 'matricula', 'id'],
      dtype='object')

In [68]:
df.shape

(309815, 57)

In [69]:
df['codigo_curso'].unique().size

6756

In [70]:
df['tp_cota'].unique()

array([None, 'PPI ', 'D   ', 'DD  ', 'PPID', 'PP  ', 'I   ', 'PPD '],
      dtype=object)

In [71]:
atributos_numerados = ['nota_l_com_peso','nota_ch_com_peso', 'nota_cn_com_peso',
        'nota_m_com_peso', 'nota_r_com_peso', 'nota_candidato',
        'qt_vagas_concorrencia', 'nota_corte']

## ENGINEERING

### notas: removendo outliers

removendo as notas consideradas _outliers_ do conjunto de dados para fazer o treinamento 

In [72]:
df_filtrado = df.copy()
notas = ['nota_l','nota_ch', 'nota_cn',
        'nota_m', 'nota_r', 'nota_candidato']


for nota in notas:
    primeiro_quartil = df[nota].quantile(0.25)
    terceiro_quartil = df[nota].quantile(0.75)
    iqr = terceiro_quartil - primeiro_quartil
    limite_inferior = primeiro_quartil - 1.5 * iqr
    limite_inferior = terceiro_quartil + 1.5 * iqr
    df_filtrado = df_filtrado[(df_filtrado[nota] >= primeiro_quartil) & (df_filtrado[nota] <= terceiro_quartil)]
df_filtrado.shape

(25987, 57)

## modalidade de concorrência: _one hot encoding_

In [85]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

teste = df_filtrado[['tipo_mod_concorrencia',  'tp_cota']]


for column in ['tipo_mod_concorrencia',  'tp_cota']:
    transformed = ohe.fit_transform(teste[column].values.reshape(-1, 1))
    ohe_df = pd.DataFrame(transformed, columns=ohe.get_feature_names_out([column]))
    teste = pd.concat([teste, ohe_df], axis=1)


ValueError: input_features should have length equal to number of features (25987), got 1

In [87]:
df.to_csv('file',index=False )

In [84]:
teste = df_filtrado[['tipo_mod_concorrencia',  'tp_cota']]
teste[column].values.reshape(-1, 1).size

25987

In [ ]:
teste

,tipo_mod_concorrencia,tp_cota,tipo_mod_concorrencia_A,tipo_mod_concorrencia_B,tipo_mod_concorrencia_L,tipo_mod_concorrencia_V,tp_cota_D,tp_cota_DD,tp_cota_I,tp_cota_PP,tp_cota_PPD,tp_cota_PPI,tp_cota_PPID,tp_cota_None,tp_cota_nan
7,A,None,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10,A,None,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
25,A,None,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,L,PPI,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
40,L,PPI,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49933,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
49934,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
49935,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
49936,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [75]:
# colunas_necessárias
ml_params = [
    'nota_l_com_peso',          #ok
    'nota_ch_com_peso',         #ok
    'nota_cn_com_peso',         #ok
    'nota_m_com_peso',          #ok
    'nota_r_com_peso',          #ok
    'nota_candidato',           #ok
    'nota_corte',               # pegar a média do curso por universidade?

#    'qt_vagas_concorrencia',    # ajudaria muito, mas tem muitos registros NaN. sem chances de usar

    # tipo_mod_concorrencia com oneHotEncoder
    'tipo_mod_concorrencia_A',
    'tipo_mod_concorrencia_B',
    'tipo_mod_concorrencia_L',
    'tipo_mod_concorrencia_V',

    'uf_campus',
    'uf_candidato',
    #    'tp_cota',
    # tp_cota com oneHotEncoder
    'tp_cota_D',
    'tp_cota_DD',
    'tp_cota_I',
    'tp_cota_PP',
    'tp_cota_PPD',
    'tp_cota_None'
]

output_desejado = 'aprovado'

In [76]:
ufs_ordenados = df[['nota_candidato', 'uf_candidato']]
ufs_ordenados

,nota_candidato,uf_candidato
0,648.55,RJ
1,647.67,RJ
2,728.01,RJ
3,568.56,SP
4,682.76,RJ
...,...,...
309810,580.45,RJ
309811,553.82,RJ
309812,631.48,RJ
309813,659.53,RJ


In [77]:
# le = OrdinalEncoder()



# fitter = le.fit()

# ufs = le.fit_transform(ufs_ordenados['uf_candidato'].values.reshape(-1, 1))
# ufs